In [1]:
import pandas as pd
import re
from sklearn import metrics
from sklearn.cluster import DBSCAN,OPTICS,Birch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *
import numpy as np
import matplotlib.pyplot as plt 
from functions import *
import math
import hdbscan
plt.style.use(plt.style.available[5])


# to ignore "DtypeWarning", generated due to reading csv files
import warnings
warnings.filterwarnings("ignore")


In [2]:
# part 3

products_DataSet = pd.read_csv('Data set/PRODUCTS.csv')

productInstance_DataSet = pd.read_csv('Data set/PRODUCTINSTANCE.csv', encoding='cp1252')

productInstance = productInstance_DataSet[['M_PRODUCT_ID',"BOOKVALUE"]].copy()
products = products_DataSet[['M_PRODUCT_ID','NAME',"VALUE"]].copy()

In [3]:

print("*****************running*******************")
ls = []
for i,record in enumerate(products['NAME']):
    s1 = re.search(' . .',record)
    s2 = re.search(' .',record)
    if s1 :
        if s1.end() == len(record) and s1.start() == len(record)-4:

            if type(products.VALUE[i]) != type(1):
                try:
                    products.VALUE[i] = pd.to_numeric(products.VALUE[i]) 
                    products.VALUE[i] /= 100000000

                except ValueError:
                    products.VALUE[i] = products.VALUE[i][:len(products.VALUE[i])-8]


    elif s2 :
        if s2.end() == len(record) and s2.start() == len(record)-2:
            if type(products.VALUE[i]) != type(1):
                try:
                    products.VALUE[i] = pd.to_numeric(products.VALUE[i]) 
                    products.VALUE[i] /= 10000

                except ValueError:
                    products.VALUE[i] = products.VALUE[i][:len(products.VALUE[i])-4]
                    products.VALUE[i] = int(products.VALUE[i])


    try:
        products.VALUE[i] = pd.to_numeric(products.VALUE[i]) 

    except ValueError:
        products.VALUE[i] = products.VALUE[i][:len(products.VALUE[i])-4]
        products.VALUE[i] = pd.to_numeric(products.VALUE[i], errors='coerce') 
    

products['VALUE'] = products['VALUE'].astype('float64')


*****************running*******************


In [4]:

merged = pd.merge(left=products,right=productInstance,how='inner',left_on='M_PRODUCT_ID',right_on='M_PRODUCT_ID')


cleanData = merged[['NAME',"VALUE",'BOOKVALUE']].dropna()

Xc = cleanData[['VALUE','BOOKVALUE']].copy()




In [5]:
X = Xc.to_numpy()

X_train_np, X_test_np = train_test_split(X,test_size=0.3,shuffle=True,random_state=40)
 # type: ignore
# X_train_np = X_train.to_numpy()
#  # type: ignore
# X_test_np = X_test.to_numpy()
 # type: ignore


In [6]:

standard = MinMaxScaler().fit(X)
train = standard.transform(X_train_np)
test = standard.transform(X_test_np)


In [7]:
from hdbscan import HDBSCAN

# print('eps=0.00070,min_samples=10')
# dbscan = DBSCAN(eps=0.000150,min_samples=3).fit(train)
# dbscan = OPTICS(min_samples=10,max_eps=0.00070).fit(train)
# dbscan = Birch(threshold=0.010,branching_factor=50).fit(train)
model = HDBSCAN(min_cluster_size=5
,cluster_selection_epsilon=0.000170
,cluster_selection_method='eom'
,prediction_data=True).fit(train)

labels = model.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

print('evaluating...')
print(f"Silhouette Coefficient: {metrics.silhouette_score(train, labels):.3f}")


Estimated number of clusters: 86
Estimated number of noise points: 242
evaluating...
Silhouette Coefficient: 0.666


In [11]:
maxBoundary = len(test)
index = []
test_ls = []
for i in range(10):
    test_ls.append(test[random_int(0,maxBoundary)])

test_ls = np.asarray(test_ls)
test_labels, strengths = hdbscan.approximate_predict(model, test_ls)

print(f"labels: {test_labels}")
print(f"soft cluster scores: {strengths}")


labels: [1080 1512 1709   61  782 1710  407 2226 2701 1689]
soft cluster scores: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
# import seaborn as sns

# figure = plt.figure(figsize=(18,18))

# pal = sns.color_palette('deep', 110)
# colors = [sns.desaturate(pal[col], sat) for col, sat in zip(model.labels_,
#                                                             model.probabilities_)]
# plt.scatter(train.T[0], train.T[1], c=colors)
